In [7]:
import os
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F

from dataset_3 import *
from dice_score import *
from model import *
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything
from pathlib import Path
from torchmetrics import Accuracy
from pytorch_lightning.loggers import WandbLogger

import random

In [5]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/detection_lung_pathology/.netrc


True

In [6]:
# code du modèle :
class mon_modele(LightningModule):
    def __init__(self, n_channels, n_classes):
        super(mon_modele, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        #code here
        self.unet = UNet(n_channels, n_classes)
        self.save_hyperparameters()

    def forward(self, x):
        #code here

        return self.unet(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self.forward(x)
        # ce_loss = nn.CrossEntropyLoss().cuda()
        # loss = ce_loss(output, y)
        output = F.softmax(output, dim=1)
        dice = multiclass_dice_coeff(output.float(), F.one_hot(y,3).permute(0, 3, 1, 2).float())
        loss =  1 - dice
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self.forward(x)
        # loss = F.cross_entropy(output, y)
        output = F.softmax(output, dim=1)
        dice = multiclass_dice_coeff(output.float(), F.one_hot(y,3).permute(0, 3, 1, 2).float())
        loss =  1 - dice
        self.log('val_loss', loss, prog_bar=True)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08)

In [15]:
def main() :

  # Using deterministic execution
  seed_everything(42)


  #declarations variables fixes :
  dir_checkpoint = Path('./checkpoint_folder/')
  dir_data = Path('./dataset/Lung_dataset_256px.hdf5')


  in_channels = 1
  out_channels = 3
  batch_size = 32
  epochs = 30


  checkpoint_callback = ModelCheckpoint(
    dirpath=dir_checkpoint,
    filename="my_model-{epoch:03d}-{val_loss:.3f}.ckpt",
    save_top_k=2,
    verbose=True,
    monitor="val_loss",
    mode="min"
  )


  # 1 Create the data loader
  #take 20% of the dataset for validation
  PatientsIdTrain,PatientsIdVal = SelectPatientsTrainVal(dir_data, 0.2)
  #If classification type  = true : datagenerator for image classification
  #If classification type  = false : datagenerator for image segmentation
  train_ds = HDF5Dataset(dir_data,PatientsIdTrain,transform=None, classification_type=False)
  val_ds = HDF5Dataset(dir_data,PatientsIdVal,transform=None,mode='valid', classification_type=False)
  n_train = len(train_ds)
  n_val = len(val_ds)
  n_classes = 3

  # 2 - Create data loaders
  # params
  loader_params = dict(batch_size=batch_size,num_workers=4,pin_memory=True,shuffle=False)
  train_dl = DataLoader(train_ds,**loader_params)
  val_dl = DataLoader(val_ds,**loader_params)


  #call the model implemented before :
  model = mon_modele(in_channels, n_classes)

  # initialise the wandb logger and name your wandb project
  wandb_logger = WandbLogger(project='IMA4202_lung_pathology_Segmentation')

  # add your batch size to the wandb config
  wandb_logger.experiment.config["batch_size"] = batch_size
  
  # Create the trainer
  trainer = Trainer(
    max_epochs=epochs,
    callbacks=checkpoint_callback,
    gpus=1,
    logger=wandb_logger,
    )

  # Train the model
  trainer.fit(model, train_dl, val_dl)
  wandb.finish()

In [ ]:
if __name__ == '__main__':
    main()

INFO:lightning_lite.utilities.seed:Global seed set to 42


patientId shape  (192,)



100%|██████████| 60/60 [00:07<00:00,  8.00it/s]


dict_keys(['A-0202_0__dic_msk', 'A-0522__dic_msk', 'A-0319__dic_msk', 'A-0537__dic_msk', 'A-0422__dic_msk', 'A-0656__dic_msk', 'A-0070__dic_msk', 'A-0074_1__dic_msk', 'A-0072__dic_msk', 'A-0044__dic_msk', 'A-0323__dic_msk', 'A-0421__dic_msk', 'A-0288__dic_msk', 'A-0285__dic_msk', 'A-0569__dic_msk', 'A-0130__dic_msk', 'A-0548__dic_msk', 'A-0299__dic_msk', 'A-0418__dic_msk', 'A-0296__dic_msk', 'A-0635__dic_msk', 'A-0570__dic_msk', 'A-0445__dic_msk', 'A-0387__dic_msk', 'A-0295__dic_msk', 'A-0435__dic_msk', 'A-0106__dic_msk', 'A-0579__dic_msk', 'A-0263__dic_msk', 'A-0531__dic_msk', 'A-0423__dic_msk', 'A-0614__dic_msk', 'A-0077__dic_msk', 'A-0361__dic_msk', 'A-0629__dic_msk', 'A-0034__dic_msk', 'A-0504__dic_msk', 'A-0534__dic_msk', 'A-0612__dic_msk', 'A-0338__dic_msk', 'A-0416__dic_msk', 'A-0590__dic_msk', 'A-0165__dic_msk', 'A-0521__dic_msk', 'A-0196_0__dic_msk', 'A-0665__dic_msk', 'A-0392__dic_msk', 'A-0247__dic_msk', 'A-0316__dic_msk', 'A-0476__dic_msk', 'A-0129__dic_msk', 'A-0575__dic_m


100%|██████████| 38/38 [00:06<00:00,  6.26it/s]


...


100%|██████████| 34/34 [00:15<00:00,  2.22it/s]


dict_keys(['A-0589__dic_msk', 'A-0215__dic_msk', 'A-0380__dic_msk', 'A-0557__dic_msk', 'A-0696__dic_msk', 'A-0264__dic_msk', 'A-0314__dic_msk', 'A-0151__dic_msk', 'A-0199__dic_msk', 'A-0500__dic_msk', 'A-0025__dic_msk', 'A-0402__dic_msk', 'A-0240__dic_msk', 'A-0638__dic_msk', 'A-0562__dic_msk', 'A-0657__dic_msk', 'A-0233__dic_msk', 'A-0331__dic_msk', 'A-0164__dic_msk', 'A-0039__dic_msk', 'A-0417__dic_msk', 'A-0366__dic_msk', 'A-0339__dic_msk', 'A-0627__dic_msk', 'A-0643__dic_msk', 'A-0360__dic_msk', 'A-0041__dic_msk', 'A-0054__dic_msk', 'A-0256_1__dic_msk', 'A-0604__dic_msk', 'A-0498__dic_msk', 'A-0073__dic_msk', 'A-0313__dic_msk', 'A-0626__dic_msk'])


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 11, global step 5376: 'val_loss' reached 0.38815 (best 0.38815), saving model to '/content/drive/MyDrive/TSP/2A/IMA4202/checkpoint_folder/my_model-epoch=011-val_loss=0.388.ckpt.ckpt' as top 2
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/event_file_writer.py", line 219, in run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/event_file_writer.py", line 69, in flush
    self._py_recordio_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/record_writer.py", line 193, in flush
    self._writer.flush()
OSError: [Errno 107] Transport endpoint is not connected
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspe

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 38, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 645, in _fit_impl
    self._run(model, ckpt_path=self.ckpt_path)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1098, in _run
    results = self._run_stage()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1177, in _run_stage
    self._run_train()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1200, in _run_train
    self.fit_loop.run()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/loop.py", line 199, in run
    self.advance(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py", line 267, in advanc